In [ ]:
import pandas as pd

from meridian import constants
from meridian.data import data_frame_input_data_builder as data_builder
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution

from mmm_eval import (
    MeridianConfig, MeridianInputDataBuilderSchema, run_evaluation)

import tensorflow_probability as tfp


## Load your data

The data below comes from the Meridian repository and gives a good example of how the input data should be formatted.

In [ ]:
df = pd.read_excel(
    'https://github.com/google/meridian/raw/main/meridian/data/simulated_data/xlsx/geo_media.xlsx',
    engine='openpyxl',
)
df.head()

A Meridian model object doesn't accept a dataframe, it instead accepts an instance of its internal `data_builder` object.
We create this object below providing the required arguments (media channels, spend columns, KPI column etc)

In [ ]:
channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4", "Channel5"]
control_cols = ["GQV", "Discount", "Competitor_Sales"]
media_cols = [f"{channel}_impression" for channel in channels]
media_spend_cols = [f"{channel}_spend" for channel in channels]

builder = (
    data_builder.DataFrameInputDataBuilder(kpi_type='non_revenue')
        .with_kpi(df, kpi_col="conversions")
        .with_revenue_per_kpi(df, revenue_per_kpi_col="revenue_per_conversion")
        .with_population(df)
        .with_controls(df, control_cols=control_cols)
)
builder = builder.with_media(
    df,
    media_cols=media_cols,
    media_spend_cols=media_spend_cols,
    media_channels=channels,
)

data = builder.build()
data

## Define a Meridian MMM

A Meridian model requires one argument - input data (in the form of a data builder object)

As an example, we also include an identical set of priors for all media channels (based on a Lognormal(0.2, 0.9) distribution).

In [4]:
roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)
mmm = model.Meridian(input_data=data, model_spec=model_spec)

# Preprocess the data for the evaluation suite

One requirement of the evaluation suite is that it expects revenue to run its tests, not revenue per KPI.

We apply this change to our dataframe below.

In [ ]:
data_preproc = df.copy()
data_preproc["revenue"] = data_preproc["revenue_per_conversion"]*data_preproc["conversions"]

# restrict to only two geos
data_preproc = data_preproc[data_preproc["geo"].isin(["Geo0"])]

# restrict to only post-2023
data_preproc = data_preproc[pd.to_datetime(data_preproc["time"]) > pd.Timestamp("2023-01-01")]
data_preproc = data_preproc[pd.to_datetime(data_preproc["time"]) < pd.Timestamp("2023-11-01")]


data_preproc.head()

# Create an `mmm-eval` config 

The `mmm-eval` package can work with a variety of frameworks. It does this by allowing you to create a config from some simple arguments. These slightly differ between frameworks, but for meridian this involves just
1. the meridian model object
2. the date column
3. the media channels
4. the media spend column names
5. the response column

In [6]:
# Create an instance of the mmm-eval Meridian Config
input_data_builder_config = MeridianInputDataBuilderSchema(
    date_column="time",
    media_channels=channels,
    channel_spend_columns=media_spend_cols,
    channel_impressions_columns=media_cols,
    response_column="conversions",
    control_columns=control_cols, #Optional but provided for example
)
input_data_builder_config

MeridianInputDataBuilderSchema(date_column='time', media_channels=['Channel0', 'Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5'], channel_spend_columns=['Channel0_spend', 'Channel1_spend', 'Channel2_spend', 'Channel3_spend', 'Channel4_spend', 'Channel5_spend'], channel_impressions_columns=['Channel0_impression', 'Channel1_impression', 'Channel2_impression', 'Channel3_impression', 'Channel4_impression', 'Channel5_impression'], channel_reach_columns=None, channel_frequency_columns=None, organic_media_columns=None, organic_media_channels=None, non_media_treatment_columns=None, response_column='conversions', control_columns=['GQV', 'Discount', 'Competitor_Sales'])

Crate the full `mmm-eval` meridian config

In [ ]:
# specify a larger number of samples if you want quality results
sample_posterior_kwargs = dict(n_chains=1, n_adapt=10, n_burnin=10, n_keep=10) #Example of setting the samples and chains if we dont want to use the defaults
config = MeridianConfig.from_model_object(mmm, 
                                          input_data_builder_config=input_data_builder_config,
                                          revenue_column="revenue", 
                                          sample_posterior_kwargs=sample_posterior_kwargs
                                          )
config

# Run the test suite

The evaluation suite requires 3 arguments
1. The framework name
2. The `mmm-eval` version of the config for that framework
3. The dataframe to run the tests on

We can also include the tests to run. If they are not included, it runs all tests by default.

## Accuracy Tests

In [9]:
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["in_sample_accuracy"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mape,mape,29.824497,False,in_sample_accuracy,2025-08-02T18:09:39.772036
1,smape,smape,26.875219,False,in_sample_accuracy,2025-08-02T18:09:39.772036
2,r_squared,r_squared,0.354487,False,in_sample_accuracy,2025-08-02T18:09:39.772036


In [8]:
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["holdout_accuracy"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mape,mape,33.966792,False,holdout_accuracy,2025-08-02T15:24:01.309671
1,smape,smape,44.510339,False,holdout_accuracy,2025-08-02T15:24:01.309671
2,r_squared,r_squared,-1.229455,False,holdout_accuracy,2025-08-02T15:24:01.309671


In [16]:
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["cross_validation"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mean_mape,mean_mape,30.036020,False,cross_validation,2025-08-03T14:45:36.873890
1,std_mape,std_mape,5.256887,False,cross_validation,2025-08-03T14:45:36.873890
2,mean_smape,mean_smape,35.014834,False,cross_validation,2025-08-03T14:45:36.873890
3,std_smape,std_smape,11.922026,False,cross_validation,2025-08-03T14:45:36.873890
4,mean_r_squared,mean_r_squared,-0.196312,False,cross_validation,2025-08-03T14:45:36.873890


## Stability Tests

In [8]:
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["refresh_stability"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mean_percentage_change,mean_percentage_change_Channel0,99.155874,False,refresh_stability,2025-08-03T15:01:18.482792
1,mean_percentage_change,mean_percentage_change_Channel1,417.025152,False,refresh_stability,2025-08-03T15:01:18.482792
2,mean_percentage_change,mean_percentage_change_Channel2,312.913243,False,refresh_stability,2025-08-03T15:01:18.482792
3,mean_percentage_change,mean_percentage_change_Channel3,223.468202,False,refresh_stability,2025-08-03T15:01:18.482792
4,mean_percentage_change,mean_percentage_change_Channel4,141.362042,False,refresh_stability,2025-08-03T15:01:18.482792
5,mean_percentage_change,mean_percentage_change_Channel5,114.704491,False,refresh_stability,2025-08-03T15:01:18.482792
6,std_percentage_change,std_percentage_change_Channel0,37.458950,False,refresh_stability,2025-08-03T15:01:18.482792
7,std_percentage_change,std_percentage_change_Channel1,638.996071,False,refresh_stability,2025-08-03T15:01:18.482792
8,std_percentage_change,std_percentage_change_Channel2,313.987235,False,refresh_stability,2025-08-03T15:01:18.482792
9,std_percentage_change,std_percentage_change_Channel3,272.016323,False,refresh_stability,2025-08-03T15:01:18.482792


In [8]:
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["perturbation"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,percentage_change,percentage_change_Channel0,95.060646,False,perturbation,2025-08-03T14:28:58.206225
1,percentage_change,percentage_change_Channel1,238.593544,False,perturbation,2025-08-03T14:28:58.206225
2,percentage_change,percentage_change_Channel2,1055.118345,False,perturbation,2025-08-03T14:28:58.206225
3,percentage_change,percentage_change_Channel3,48.323219,False,perturbation,2025-08-03T14:28:58.206225
4,percentage_change,percentage_change_Channel4,116.035639,False,perturbation,2025-08-03T14:28:58.206225
5,percentage_change,percentage_change_Channel5,148.851493,False,perturbation,2025-08-03T14:28:58.206225


In [8]:
result = run_evaluation(framework="meridian", config=config, data=data_preproc, test_names=["placebo"])
result

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,shuffled_channel_roi,shuffled_channel_roi_Channel0_shuffled,-7.633817,False,placebo,2025-08-02T18:08:18.885909


In [3]:
# result = run_evaluation(framework="meridian", config=config, data=data_preproc)
# result